In [1]:
import requests
import pandas as pd
import json
import mysql.connector
from datetime import datetime, timedelta, timezone
import time
import nvda_config
import twelvedata as td
from twelvedata import TDClient


In [2]:
# set up login
Cpassword = nvda_config.mysql_password
td = TDClient(apikey=nvda_config.twelve_api_nvda) 
base_url = 'https://api.twelvedata.com'


In [7]:
# Estab;ish MySQL object
mydb = mysql.connector.connect(
    host="localhost",   
    user="root",         
    password=Cpassword,  
    database="nvdav1"   
)

mycursor = mydb.cursor()




In [9]:


# Twelve Data API parameters
symbol = "NVDA"
interval = "1min"
outputsize = 5000 

# Initialize a counter for the number of requests
request_count = 0

while request_count < 390:
    # Fetch time series data and indicators
    price_pdf = td.price(
        symbol=symbol
    )
    # Convert to pandas Dataframes
    price_json = price_pdf.as_json()
    price_df = pd.DataFrame(price_json, index=["price"])

    # Prepare for database insertion
    time_series_data = []

    # Process and align data
    for index, row in price_df.iterrows():
        # Generate the current datetime string
        datetime_str = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

        time_series_data.append(
            (
                symbol,
                datetime_str,
                row["price"]
            )
        )

    # Insert tick data into MySQL
    sql_time_series = """
    INSERT INTO nvda_new (symbol, datetime, price)
    VALUES (%s, %s, %s)
    """
    mycursor.executemany(sql_time_series, time_series_data)
    mydb.commit()

    # Increment the request counter
    request_count += 1
    print("Requests elapsed: ", request_count, " at time:", datetime.now())
    # Sleep for a short period before fetching the next data
    time.sleep(59.75)  # Adjust the sleep time as needed

print("Program completed after 390 requests.")


Requests elapsed:  1  at time: 2024-07-26 09:54:21.446472
Requests elapsed:  2  at time: 2024-07-26 09:55:22.093299
Requests elapsed:  3  at time: 2024-07-26 09:56:22.697053
Requests elapsed:  4  at time: 2024-07-26 09:57:23.240963
Requests elapsed:  5  at time: 2024-07-26 09:58:23.765052
Requests elapsed:  6  at time: 2024-07-26 09:59:24.338460
Requests elapsed:  7  at time: 2024-07-26 10:00:24.964067
Requests elapsed:  8  at time: 2024-07-26 10:01:25.791742
Requests elapsed:  9  at time: 2024-07-26 10:02:26.282492
Requests elapsed:  10  at time: 2024-07-26 10:03:26.716475
Requests elapsed:  11  at time: 2024-07-26 10:04:27.369417
Requests elapsed:  12  at time: 2024-07-26 10:05:27.704448
Requests elapsed:  13  at time: 2024-07-26 10:06:28.067458
Requests elapsed:  14  at time: 2024-07-26 10:07:28.598746
Requests elapsed:  15  at time: 2024-07-26 10:08:29.236284
Requests elapsed:  16  at time: 2024-07-26 10:09:29.848069
Requests elapsed:  17  at time: 2024-07-26 10:10:30.551143
Reques

KeyboardInterrupt: 